In [1]:
import pandas as pd
import re
from pathlib import Path
import json

#### functions for ultraplex statistics extraction ####
def extract_nread_with_barcode(s):
    ''' extract nread with barcode from ultraplex.log'''
    pattern = r"(\d+)\s+\((\d+\.\d+)%\)\s+reads\s+correctly\s+assigned"
    match = re.search(pattern, s)
    if match:
        num_reads = match.group(1)
        percentage = match.group(2)
        return num_reads, percentage
    else:
        print(s)
        return None, None
def extract_input_reads(s):
    '''Demultiplexing complete! 21532666 reads processed in 1095.0 seconds'''
    pattern = r'(?<=Demultiplexing complete! )\d+(?= reads processed)'
    
    matches = re.findall(pattern, s)

    if matches:
        num_reads = int(matches[0])
        
        return num_reads
    else:
        print(s)
        return None

def extract_quality_trimmed(s):
    pattern = r"(\d+)\s+\((\d+\.\d+)%\)\s+reads\s+quality\s+trimmed"
    match = re.search(pattern, s)
    if match:
        num_reads = match.group(1)
        percentage = match.group(2)
        return num_reads, percentage
    else:
        print(s)
        return None, None

def get_ultraplex_stat(basedir):
    ultraplex_stat = []
    for file in basedir.glob('*/fastqs/ultraplex*.log'):
        libname = file.parent.parent.name
        with open(file) as f:
            lines = f.readlines()
            barcode_nread, barcode_perc = extract_nread_with_barcode(lines[-1])
            ultraplex_input_nread = extract_input_reads([l for l in lines if 'Demultiplex' in l][0])
        ultraplex_stat.append([libname, barcode_nread, barcode_perc, ultraplex_input_nread])
    ultraplex_stat = pd.DataFrame(ultraplex_stat, columns = ['libname', 'nread_w_barcode', '%barcode', 'input_to_ultraplex']
                        ).drop_duplicates('libname').set_index('libname')
    return ultraplex_stat

def get_fastp_stat(basedir):
    ''' extracts fastp statistics '''
    fastp_stats = []
    for fastp_stat_file in basedir.glob('QC/*umi.json'):
        fastp_stat = json.load(open(fastp_stat_file))
        fastp_stats.append([fastp_stat_file.name.split('.')[0],
                        fastp_stat['summary']['before_filtering']['total_reads'],
                        fastp_stat['summary']['after_filtering']['total_reads']]
                        )
    fastp_stats = pd.DataFrame(fastp_stats, columns = ['libname', 'before_umi_polyG', 'after_umi_polyG']).set_index('libname')
    fastp_stats['%polyG or too short']=100*(fastp_stats['before_umi_polyG']-fastp_stats['after_umi_polyG'])/fastp_stats['before_umi_polyG']
    return fastp_stats



In [2]:
basedir = Path('/tscc/projects/ps-yeolab3/bay001/for_charlene/scratch/sra_test')

In [3]:
# read inputs:
cutadapt = pd.read_csv(basedir/'QC/cutadapt_stat.csv', index_col = 0)
genome_stat = pd.read_csv(basedir/'QC/mapping_stats.csv', index_col = 0)
dup_df = pd.read_csv(basedir/'QC/dup_level.csv', index_col = 0)
ultraplex_stat = get_ultraplex_stat(Path('.'))
fastp_stats = get_fastp_stat(Path('.'))

# generate mapping summary
genome_stat['libname']=genome_stat['STAR Log filename'].str.split('/', expand = True)[0]
mapping_summary = pd.concat([genome_stat.groupby(by = 'libname')['Uniquely mapped reads number'].sum(),
        genome_stat.groupby(by = 'libname')['Number of reads mapped to multiple loci'].sum(),
                            genome_stat.groupby(by = 'libname')['Number of input reads'].sum(),
        ], axis = 1
        )
mapping_summary['total_mapped']=mapping_summary[['Uniquely mapped reads number', 'Number of reads mapped to multiple loci']].sum(axis = 1)
mapping_summary['%Unique map']=100*mapping_summary['Uniquely mapped reads number']/mapping_summary['Number of input reads']
mapping_summary['%Multi map']=100*mapping_summary['Number of reads mapped to multiple loci']/mapping_summary['Number of input reads']
mapping_summary['% mapped']=100*mapping_summary['total_mapped']/mapping_summary['Number of input reads']

# generate dedup summary: is the sum of read 1 and read2
dup_df['libname']=dup_df['dup_bam'].str.split('/', expand = True)[0]
dup_summary_df=dup_df.groupby(by = ['libname'])['before_dedup', 'after_dedup'].sum()
dup_summary_df['% Unique frag']=100*dup_summary_df['after_dedup']/dup_summary_df['before_dedup']

# generate repeat counts summary
repeat_cnt_stat = []
for repeat_cnt_file in list((basedir/'counts/repeats/megatables/class').glob('*tsv.gz')):
    repeat_cnt = pd.read_csv(repeat_cnt_file, sep = '\t', index_col = 0)
    n_read_in_repeat = repeat_cnt.sum().sum()
    n_read_in_rRNA = repeat_cnt.loc['rRNA'].sum()
    
    repeat_cnt_stat.append([repeat_cnt_file.name.split('.')[0],
                            n_read_in_repeat,
                            n_read_in_rRNA]
                        )
repeat_cnt_stat = pd.DataFrame(repeat_cnt_stat, columns = ['libname', 'n_read_in_repeat', 'n_read_in_rRNA']).set_index('libname')

# generate genome counts summary
reads_in_window = []
for region_cnt in list(Path(basedir/'QC/read_count').glob('*region.csv')):
    region_read_count = pd.read_csv(region_cnt, index_col = 0)
    reads_in_window.append([region_cnt.name.split('.')[0], region_read_count.sum().sum()])
reads_in_window = pd.DataFrame(reads_in_window, columns = ['libname', 'nread_in_genome_window']).set_index('libname')

# make summary
cutadapt.index = [i.split('.')[0] for i in cutadapt.index]
try:
    summary = pd.concat([cutadapt[['Total reads processed', '% Reads that were too short']],
            fastp_stats,
            ultraplex_stat,

            mapping_summary,
            dup_summary_df,

            reads_in_window,
                        repeat_cnt_stat],
            axis = 1)
    is_paired = False
except:
    summary = pd.concat([cutadapt[['Total read pairs processed', '% Pairs that were too short']],
            fastp_stats,
            ultraplex_stat,

            mapping_summary,
            dup_summary_df,

            reads_in_window,
                        repeat_cnt_stat],
            axis = 1)
    is_paired = True

if not is_paired:
    summary['%_in_genome_window']=100*summary['nread_in_genome_window']/summary['after_dedup']
    summary['%_in_repeat']=100*summary['n_read_in_repeat']/summary['after_dedup']
    summary['%_in_rRNA']=100*summary['n_read_in_rRNA']/summary['after_dedup']
else:
    # counting is only counting 1 read, but samtools counts both
    summary['%_in_genome_window']=100*2*summary['nread_in_genome_window']/summary['after_dedup']
    summary['%_in_repeat']=100*2*summary['n_read_in_repeat']/summary['after_dedup']
    summary['%_in_rRNA']=100*2*summary['n_read_in_rRNA']/summary['after_dedup']



/tscc/nfs/home/hsher/miniconda3/envs/my_metadensity/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [4]:
summary

,Total reads processed,% Reads that were too short,before_umi_polyG,after_umi_polyG,%polyG or too short,nread_w_barcode,%barcode,input_to_ultraplex,Uniquely mapped reads number,Number of reads mapped to multiple loci,...,% mapped,before_dedup,after_dedup,% Unique frag,nread_in_genome_window,n_read_in_repeat,n_read_in_rRNA,%_in_genome_window,%_in_repeat,%_in_rRNA
K562_rep4,236070272,10.4,NaN,NaN,NaN,NaN,NaN,NaN,36119518.0,144887779.0,...,90.642654,181007297,11655796,6.439407,7740556,3380094,574876,66.409501,28.999255,4.932104
K562_rep6,316340455,3.5,NaN,NaN,NaN,NaN,NaN,NaN,45723494.0,220340909.0,...,91.142603,266064403,8281139,3.112457,4996487,2800053,573098,60.335746,33.812414,6.920521
